In [26]:
import pandas as pd
from utils import *
from typing import Any

<h1>Cleaning a CSV</h1>

Read in a CSV

In [27]:
# load a dataframe
infile_path = "../data/raw/silver_shelby_county_properties_municipality_breakdown - s5_2022_iph.csv"
df = pd.read_csv(infile_path)
df

,Owner,Location,PILOT Parcel No,Underlying Parcel Number,Begin Date,End Date,Assmt Taxes,PILOT Contract Amt,PILOT Debt Service Amt,Difference
0,ST PETER'S MANOR,108 AUBURNDALE,IPH0000A000000,1704800000290,09/26/1977,10/01/2048,"$128,377","$6,792",$0,"-$121,585"
1,HIGHLAND MEADOWS APTS,3517 ANDY WAY,IPH0000B000000,0830200000017C,10/16/1989,12/01/2029,"$122,950",$89,$0,"-$122,861"
2,COUNTRYSIDE NORTH APARTMENTS L.P.,6920 REESE,IPH0000D000000,9510000000750,12/01/1998,12/01/2038,"$91,035","$26,512",$0,"-$64,523"
3,APARTMENTS AT LAPALOMA LTD,0 PARKWAY,IPH0000F000000,0310940000003C,12/01/1998,12/01/2029,"$28,646","$3,513",$0,"-$25,133"
4,THE TREZEVANT EPISCOPAL HOME,0 WAYNOKA,IPH0001S000000,"04404300000690, 04404300000700, 04404300000710",03/28/2006,03/28/2034,"$536,553","$47,460",$0,"-$489,093"
5,"WESLEY GRACELAND GARDENS, L.P.",1445 RAINES,IPH0001U000000,"07900400000060, 07900400000090, 07900400000100",08/01/2016,08/23/2033,"$127,867","$4,230",$0,"-$123,637"


In [28]:
# Clean the column headers
df = clean_headers(df, [remove_special_for_words, truncate, snake_case, lower_case])
df.columns

Index(['owner', 'location', 'pilot_parcel_no', 'underlying_parcel_number',
       'begin_date', 'end_date', 'assmt_taxes', 'pilot_contract_amt',
       'pilot_debt_service_amt', 'difference'],
      dtype='object')

In [29]:
# Clean the string columns
string_columns = ["owner", 
                  "location"]
df = clean_columns(df, string_columns, [remove_special_for_words, truncate, lower_case])
df

,owner,location,pilot_parcel_no,underlying_parcel_number,begin_date,end_date,assmt_taxes,pilot_contract_amt,pilot_debt_service_amt,difference
0,st peter s manor,108 auburndale,IPH0000A000000,1704800000290,09/26/1977,10/01/2048,"$128,377","$6,792",$0,"-$121,585"
1,highland meadows apts,3517 andy way,IPH0000B000000,0830200000017C,10/16/1989,12/01/2029,"$122,950",$89,$0,"-$122,861"
2,countryside north apartments l p,6920 reese,IPH0000D000000,9510000000750,12/01/1998,12/01/2038,"$91,035","$26,512",$0,"-$64,523"
3,apartments at lapaloma ltd,0 parkway,IPH0000F000000,0310940000003C,12/01/1998,12/01/2029,"$28,646","$3,513",$0,"-$25,133"
4,the trezevant episcopal home,0 waynoka,IPH0001S000000,"04404300000690, 04404300000700, 04404300000710",03/28/2006,03/28/2034,"$536,553","$47,460",$0,"-$489,093"
5,wesley graceland gardens l p,1445 raines,IPH0001U000000,"07900400000060, 07900400000090, 07900400000100",08/01/2016,08/23/2033,"$127,867","$4,230",$0,"-$123,637"


In [30]:
df.columns

Index(['owner', 'location', 'pilot_parcel_no', 'underlying_parcel_number',
       'begin_date', 'end_date', 'assmt_taxes', 'pilot_contract_amt',
       'pilot_debt_service_amt', 'difference'],
      dtype='object')

In [31]:
# Clean the numeric columns
numeric_columns = ['assmt_taxes', 'pilot_contract_amt',
       'pilot_debt_service_amt', 'difference']
df = clean_columns(df, numeric_columns, [remove_special_for_numbers, convert_only_dash_to_missing])

df

,owner,location,pilot_parcel_no,underlying_parcel_number,begin_date,end_date,assmt_taxes,pilot_contract_amt,pilot_debt_service_amt,difference
0,st peter s manor,108 auburndale,IPH0000A000000,1704800000290,09/26/1977,10/01/2048,128377,6792,0,-121585
1,highland meadows apts,3517 andy way,IPH0000B000000,0830200000017C,10/16/1989,12/01/2029,122950,89,0,-122861
2,countryside north apartments l p,6920 reese,IPH0000D000000,9510000000750,12/01/1998,12/01/2038,91035,26512,0,-64523
3,apartments at lapaloma ltd,0 parkway,IPH0000F000000,0310940000003C,12/01/1998,12/01/2029,28646,3513,0,-25133
4,the trezevant episcopal home,0 waynoka,IPH0001S000000,"04404300000690, 04404300000700, 04404300000710",03/28/2006,03/28/2034,536553,47460,0,-489093
5,wesley graceland gardens l p,1445 raines,IPH0001U000000,"07900400000060, 07900400000090, 07900400000100",08/01/2016,08/23/2033,127867,4230,0,-123637


In [32]:
# Clean the date columns

# Because this data has an idiosyncratic "end date" value of "End of Loan", 
# we must define a dataset-specific function
def cast_end_date_to_datetime(entry: str | Number):
    try:
        return pd.to_datetime(str(entry))
    except ValueError:
        return entry
df["begin_date"] = pd.to_datetime(df["begin_date"])
df["end_date"] = df["end_date"].apply(lambda x: cast_end_date_to_datetime(x))
df

,owner,location,pilot_parcel_no,underlying_parcel_number,begin_date,end_date,assmt_taxes,pilot_contract_amt,pilot_debt_service_amt,difference
0,st peter s manor,108 auburndale,IPH0000A000000,1704800000290,1977-09-26,2048-10-01,128377,6792,0,-121585
1,highland meadows apts,3517 andy way,IPH0000B000000,0830200000017C,1989-10-16,2029-12-01,122950,89,0,-122861
2,countryside north apartments l p,6920 reese,IPH0000D000000,9510000000750,1998-12-01,2038-12-01,91035,26512,0,-64523
3,apartments at lapaloma ltd,0 parkway,IPH0000F000000,0310940000003C,1998-12-01,2029-12-01,28646,3513,0,-25133
4,the trezevant episcopal home,0 waynoka,IPH0001S000000,"04404300000690, 04404300000700, 04404300000710",2006-03-28,2034-03-28,536553,47460,0,-489093
5,wesley graceland gardens l p,1445 raines,IPH0001U000000,"07900400000060, 07900400000090, 07900400000100",2016-08-01,2033-08-23,127867,4230,0,-123637


In [33]:
# Clean the list columns

# Pandas always reads in lists as strings, so they will ultimately be converted to a string type, but putting them in lists firsts will allow us to convert them back to lists more quickly the next time we read them in because the list brackets will be included 
# in the output string

list_columns = ["pilot_parcel_no", "underlying_parcel_number"]
for col in list_columns:
    df[col] = df[col].apply(lambda x: str(x).split(","))
df

,owner,location,pilot_parcel_no,underlying_parcel_number,begin_date,end_date,assmt_taxes,pilot_contract_amt,pilot_debt_service_amt,difference
0,st peter s manor,108 auburndale,[IPH0000A000000],[1704800000290],1977-09-26,2048-10-01,128377,6792,0,-121585
1,highland meadows apts,3517 andy way,[IPH0000B000000],[0830200000017C],1989-10-16,2029-12-01,122950,89,0,-122861
2,countryside north apartments l p,6920 reese,[IPH0000D000000],[9510000000750],1998-12-01,2038-12-01,91035,26512,0,-64523
3,apartments at lapaloma ltd,0 parkway,[IPH0000F000000],[0310940000003C],1998-12-01,2029-12-01,28646,3513,0,-25133
4,the trezevant episcopal home,0 waynoka,[IPH0001S000000],"[04404300000690, 04404300000700, 04404300000...",2006-03-28,2034-03-28,536553,47460,0,-489093
5,wesley graceland gardens l p,1445 raines,[IPH0001U000000],"[07900400000060, 07900400000090, 07900400000...",2016-08-01,2033-08-23,127867,4230,0,-123637


In [34]:
# Create a unique index column based on the values in the identifying column
# Note: Ids are based on a deterministic hash, which means they are 
# reproducible even if the function is run by a different user at a different time,
# as long as the string in the identifying column is the same. 

identifying_columns = ["owner"]
index_name = "owner_id"
df = set_unique_index(df=df, columns_to_hash = identifying_columns, index_length_limit=12, index_name=index_name)
df

,owner,location,pilot_parcel_no,underlying_parcel_number,begin_date,end_date,assmt_taxes,pilot_contract_amt,pilot_debt_service_amt,difference
owner_id,,,,,,,,,,
051a018be14c,st peter s manor,108 auburndale,[IPH0000A000000],[1704800000290],1977-09-26,2048-10-01,128377,6792,0,-121585
87106a47fd39,highland meadows apts,3517 andy way,[IPH0000B000000],[0830200000017C],1989-10-16,2029-12-01,122950,89,0,-122861
5cf3e3c81542,countryside north apartments l p,6920 reese,[IPH0000D000000],[9510000000750],1998-12-01,2038-12-01,91035,26512,0,-64523
fe50ebe8a3cc,apartments at lapaloma ltd,0 parkway,[IPH0000F000000],[0310940000003C],1998-12-01,2029-12-01,28646,3513,0,-25133
728ee4fde2a1,the trezevant episcopal home,0 waynoka,[IPH0001S000000],"[04404300000690, 04404300000700, 04404300000...",2006-03-28,2034-03-28,536553,47460,0,-489093
ed07cbec1011,wesley graceland gardens l p,1445 raines,[IPH0001U000000],"[07900400000060, 07900400000090, 07900400000...",2016-08-01,2033-08-23,127867,4230,0,-123637


In [35]:
# Check the data types of each variable
df.dtypes

owner                               object
location                            object
pilot_parcel_no                     object
underlying_parcel_number            object
begin_date                  datetime64[ns]
end_date                    datetime64[ns]
assmt_taxes                         object
pilot_contract_amt                  object
pilot_debt_service_amt              object
difference                          object
dtype: object

In [36]:
# Fix the data type of each variable

names_to_types = {'owner': 'string',
                  'location': 'string',
                  'pilot_parcel_no': 'string', 
                  'underlying_parcel_number': 'string',
                  'begin_date': 'datetime64[ns]', 
                  'end_date': 'object',
                  'assmt_taxes': 'float64',
                  'pilot_contract_amt': 'float64',
       'pilot_debt_service_amt': 'float64', 
                  'difference': 'float64'}
df = cast_data_types(df, names_to_types)
df.dtypes

owner                       string[python]
location                    string[python]
pilot_parcel_no             string[python]
underlying_parcel_number    string[python]
begin_date                  datetime64[ns]
end_date                            object
assmt_taxes                        float64
pilot_contract_amt                 float64
pilot_debt_service_amt             float64
difference                         float64
dtype: object

In [37]:
# Save the now-clean csv and convert it to utf-8 format
outfile_path = "../data/processed/s5_2022_iph.csv"
df.to_csv(outfile_path)
convert_utf8(original_file_path=outfile_path, new_file_path=outfile_path)